In [3]:
import json
import sqlite3 as sql

class DB:
    def __init__(self):
        self.db_name = 'roster.sqlite'
        self.con = ''
        self.cur = ''
        self.jfile = ''

        self.fname = input('Enter file name: ')
        if len(self.fname) < 1:
            self.fname = 'roster_data.json'

        self.jfile = open(self.fname).read()
        self.jfile = json.loads(self.jfile)

    def db_connection(self):
        self.con = sql.connect(self.db_name)
        self.cur = self.con.cursor()

    def create_db(self):
        self.cur.executescript('''
            DROP TABLE IF EXISTS User;
            DROP TABLE IF EXISTS Member;
            DROP TABLE IF EXISTS Course;
            CREATE TABLE User(
                id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                name TEXT UNIQUE
            );
            CREATE TABLE Course(
                id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                title TEXT UNIQUE
            );
            CREATE TABLE Member(
                user_id INTEGER,
                course_id INTEGER,
                role INTEGER,
                PRIMARY KEY(user_id, course_id)
            );
        ''')

    def add_db(self):
        for entry in self.jfile:
            name = entry[0]
            title = entry[1]
            role = entry[2]

            #print(name, title, role)

            self.cur.execute('''INSERT OR IGNORE INTO User(name)
                            VALUES(?)''', (name,))
            self.cur.execute('SELECT id FROM User WHERE name = ?', (name,))
            user_id = self.cur.fetchone()[0]

            self.cur.execute('''INSERT OR IGNORE INTO Course(title)
                            VALUES(?)''', (title,))
            self.cur.execute('SELECT id FROM Course WHERE title = ?', (title,))
            course_id = self.cur.fetchone()[0]

            self.cur.execute('''INSERT OR REPLACE INTO Member(user_id, course_id, role)
                            VALUES(?,?,?)''', (user_id, course_id, role))            
            
            self.con.commit()
            

    def show_db(self):
        self.showData = self.cur.execute('''
SELECT 'XYZZY' || hex(User.name || Course.title || Member.role ) AS X FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY X LIMIT 1;

        ''')

        print('########################')
        for self.data in self.showData:
            print(self.data)

    def db_disconnection(self):
        self.cur.close()


if __name__ == '__main__':
    obj = DB()
    obj.db_connection()
    obj.create_db()
    obj.add_db()
    obj.show_db()
    obj.db_disconnection()

Enter file name: roster_data.json
########################
('XYZZY4161726161736933333430',)


In [5]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

api_key = False
# If you have a Google Places API key, enter it here
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("where.data")
count = 0
for line in fh:
    if count > 200 :
        print('Retrieved 200 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database ",address)
        continue
    except:
        pass

    parms = dict()
    parms["address"] = address
    if api_key is not False: parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Failure To Retrieve ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(5)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")


FileNotFoundError: [Errno 2] No such file or directory: 'where.data'